In [ ]:
eeg_signal = []
valence = []
arousal = []
dominance = []
signal_freq = []
eeg_sig = []
gc.collect()

for person in range(2,3):
  print('Person No.' + str(person))


  address = input_path+'s'+str(person)+'.dat'

  with open(address, 'rb') as file:
    data = pkl.load(file, encoding = 'latin1')

  eeg = data['data']
  label = data['labels']

  # Assigning classes
  label[label<5] = 0
  label[(label>=5) & (label<=9)] = 1

  val = label.T[0] # Valence label
  aro = label.T[1] # Arousal label
  dom = label.T[2] # Dominance label

  del data, label


  for i in range(40): # Iterating through 40 vidoes/trials

    sig = eeg[i]
    sig = sig[:32, 384:]
    #sig = baseline_remove(sig)
    dfs = []
    for j in range(32):
      ## Dividing Delta Band
      num, den = signal.butter(4, bands['delta'], 'bandpass')
      dfs.append(signal.filtfilt(num, den, sig[j,:]))
      ## Dividing Theta Band
      num, den = signal.butter(4, bands['theta'], 'bandpass')
      dfs.append(signal.filtfilt(num, den, sig[j,:]))
      ## Dividing Alpha Band
      num, den = signal.butter(4, bands['alpha'], 'bandpass')
      dfs.append(signal.filtfilt(num, den, sig[j,:]))
      ## Dividing Beta Band
      num, den = signal.butter(4, bands['beta'], 'bandpass')
      dfs.append(signal.filtfilt(num, den, sig[j,:]))
      ## Dividing Gamma Band
      num, den = signal.butter(4, bands['gamma'], 'bandpass')
      dfs.append(signal.filtfilt(num, den, sig[j,:]))





    sig = np.array(dfs)
   #print(sig.shape)
    #sig = sig.reshape([-1,7680])
    eeg_signal.append(sig)
  del dfs, sig, eeg
  eeg_signal = np.array(eeg_signal)
  print(eeg_signal.shape)
  #eeg_signal = np.reshape(eeg_signal,[-1,32,7680])
  gc.collect()
  for i in range(40):
    v = val[i]
    a = aro[i]
    d = dom[i]
    start = 0
    # Segmenting into 2 seconds (256 timesteps) windows with no overlap
    while start + window_size <=eeg_signal.shape[2]:
      for j in range(eeg_signal.shape[1]):
        eeg_sig.append(eeg_signal[i, j, start:start+window_size])#.mean())#axis=1))
      valence.append(v)
      arousal.append(a)
      dominance.append(d)
      start += skip
eeg_sig = np.array(eeg_sig)
gc.collect()
del eeg_signal
print(eeg_sig.shape)
eeg_sig = np.reshape(eeg_sig,[-1,32*5,256])
print(eeg_sig.shape)
data = np.asarray(eeg_sig, dtype = np.float32) # Using 32 bit floating point value to save memory
del eeg_sig
valence = np.asarray(valence, dtype = np.int8)
arousal = np.asarray(arousal, dtype = np.int8)
dominance = np.asarray(dominance, dtype = np.int8)

print(data.shape)
print(valence.shape, valence[valence == 0].shape, valence[valence == 1].shape)
print(arousal.shape, arousal[arousal == 0].shape, arousal[arousal == 1].shape)
print(dominance.shape, dominance[dominance == 0].shape, dominance[dominance == 1].shape)

valence = np_utils.to_categorical(valence)
arousal = np_utils.to_categorical(arousal)
dominance = np_utils.to_categorical(dominance)